Hola de nuevo. Espero que estés listo/lista para sumergirte en una emocionante sesión de codificación, ya que eso es lo que nos espera ahora. Si bien la limpieza de datos es esencial para realizar un análisis sólido, también puede resultar tediosa y, en ocasiones, desafiante. En este punto, pondremos a prueba nuestra creatividad y habilidades en python para llevar a cabo las tareas adecuadas en nuestras bases de datos y que los resultados no solo sean fiables, sino tambien uniformes.

Comencemos con unos ejercicios de calentamiento en la base de datos de Brasil, la cual se especializa en datos turísticos de interés, centrándonos específicamente en la región de Río de Janeiro.

In [1]:
#instalaremos algunas librerias de antemano que nos serviran para mas adelante 
!pip install pycountry
!pip install wbgapi
!pip install scipy

import pycountry #funciona para conocer los nombres de los paises
import numpy as np #permite realizar operaciones aritmeticas con mayor facilidad
import wbgapi as wb #api del banco mundial
import pandas as pd #libreria para procesamiento de datos
from scipy import spatial #libreria para procesamiento de datos geograficos 

In [2]:
Datos = pd.read_csv("https://raw.githubusercontent.com/CesaRepositorio/LibroPython/main/DatosPaises/Brasil/listings.csv")

In [3]:
#Veamos la cantidad de datos vacios en cada campo, y los campos disponibles
#en este caso podemos ver el id del hotel, el nombre del hotel y del arrendatario, el grupo del barrio (casi todos vacios) 
# y el barrio, la latitud la longitud, el tipo de cuarto, el precio del cuarto, el numero de noches, numero de reviews
#entre algunas otras

Datos.isna().sum()

id                                    0
name                                 18
host_id                               0
host_name                             3
neighbourhood_group               30476
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                        7504
reviews_per_month                  7504
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                           30476
dtype: int64

In [4]:
#esta base de datos, viene por defecto muy bien arreglada y uniformizada, sin embargo, gran parte del tiempo no sera asi
#como veremos mas adelante, por lo que podemos ejecutar ejercicios sencillos para calentar y entender que podemos hacer con las herramientas

#inicialmente podemos manejar vacios, en este caso, tiene sentido que podamos llenar los vacios con informacion como no identificado
#ya que no afecta en lo absoluto el desarrollo del estudio
Datos.neighbourhood_group = Datos.neighbourhood_group.fillna("No identificado")
Datos.neighbourhood_group

0        No identificado
1        No identificado
2        No identificado
3        No identificado
4        No identificado
              ...       
30471    No identificado
30472    No identificado
30473    No identificado
30474    No identificado
30475    No identificado
Name: neighbourhood_group, Length: 30476, dtype: object

In [5]:
#tambien podemos filtrar la base en base a condiciones, en este caso los hospedajes que cuesten menos de 100 

#estos cambios no se mantienen en la base de datos, a menos de que la sobreescribas (Datos = Datos[Datos.price<=100]) o
#ingreses un nombre diferente a la variable

Datos[Datos.price<=100]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
2,200568,30m of Ipa Beach e fins de semana special price !,980805,Henrique,No identificado,Ipanema,-22.985860,-43.194110,Entire home/apt,100,30,198,2023-02-13,1.45,6,59,9,NaN
81,98969,Quartos na praia mais linda do Rio,521613,Cyro,No identificado,Recreio dos Bandeirantes,-23.031540,-43.474370,Private room,95,2,29,2023-02-18,0.21,1,165,23,NaN
98,107469,Quarto confortável com vista panorâmica,556738,Casa Da Carmen E,No identificado,Santa Teresa,-22.918890,-43.180380,Private room,61,30,2,2018-12-21,0.02,5,271,0,NaN
102,409183,"Lapa:culture, work and fun in Rio",396108,Gisella,No identificado,Santa Teresa,-22.914240,-43.180590,Entire home/apt,76,4,43,2023-03-04,0.35,2,36,17,NaN
109,273363,Lindo quarto sossegado para temporada,556738,Casa Da Carmen E,No identificado,Santa Teresa,-22.918250,-43.178720,Private room,80,30,4,2017-12-04,0.04,5,169,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30428,855624594723996017,House with balcony and sea view,437299567,Raquel,No identificado,Copacabana,-22.980909,-43.193973,Entire home/apt,95,3,0,NaN,NaN,1,336,0,NaN
30436,855836808269778455,Quarto-jardim do Flamengo,10262542,Martha,No identificado,Flamengo,-22.932953,-43.175238,Private room,83,1,0,NaN,NaN,2,179,0,NaN
30449,856337214710629509,Apartamento bem localizado,309516548,Pedro,No identificado,Campo Grande,-22.883129,-43.548928,Entire home/apt,42,10,0,NaN,NaN,1,364,0,NaN
30461,856482448445233912,"Quarto Privativo, Centro RJ",499354177,Igor,No identificado,Cidade Nova,-22.911540,-43.209187,Private room,80,1,0,NaN,NaN,3,364,0,NaN


In [6]:
#tambien podemos ver el tipo de elementos unicos que hay en cada columna
Datos["room_type"].unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [7]:
#ademas, ejecutar comandos que modifiquen estos elementos mediante funciones 
#en este caso, cambiaremos los datos de estas categorias por sus traducciones en español
Cambios = {'Entire home/apt':"Apartamento",
'Private room':"Cuarto Privado",
'Shared room':"Cuarto Compartido",
"Hotel room":"Cuarto de hotel"}

Datos["room_type"] = Datos["room_type"].apply(lambda x: Cambios[x])

Este es un pequeño repaso de lo que veremos a continuacion, por lo que no te intimides por la cantidad de datos que puedes visualizar a continuacion, ya que lo siguiente que veras son algunas variaciones y complementos de los comandos presentados arriba, de tal manera, empezamos 

In [8]:
# nuestro objetivo es el siguiente:

# En base a los datos mundiales, necesitamos encontrar un conjunto de paises optimos similares a colombia en los que se puedan
# replicar el modelo de negocios actual (empresa de transportes) en base a 4 criterios


# 1. Lugares mas contaminados por transportes (mas mejor) contaminacion por persona no contaminacion general. 
#                                                         tener mas contaminacion por que hay mas gente no es buen criterio 
# 2. Cantidad de turistas (mas mejor)
# 3. ingresos por turista (mas mejor)
# 4.  la inflacion general del pib (menos mejor)

#Para 1
#Datos de contaminacion por combustible fossil (EN.ATM.CO2E.LF.KT)
#Datos de total de poblacion (SP.POP.TOTL)

#Para 2 
#Datos de turistas que llegan (ST.INT.ARVL)

#Para 3 
#Datos de turistas que llegan (ST.INT.ARVL)
#Datos de ganancias por viaje (ST.INT.TVLR.CD,ST.INT.TVLX.CD) Debo revisar cual es el adecuado (es este ST.INT.TVLR.CD que hace referencia a ingresos de capital en el pais)

#Para 4
#Usar la inflacion calculada mediante la relacion entre el pib de la moneda loca y la moneda fija 

In [9]:
#Cargar los datos del banco mundial 
EmisionesCo2PorCombustibleLiquido = wb.data.DataFrame(["EN.ATM.CO2E.LF.KT"])
PoblacionTotal = wb.data.DataFrame(["SP.POP.TOTL"])
CantidadDeTuristas = wb.data.DataFrame(["ST.INT.ARVL"])
IngresosPorViajesTuristicos = wb.data.DataFrame(["ST.INT.TVLR.CD"])
InflacionPib = wb.data.DataFrame(["NY.GDP.DEFL.KD.ZG"])

In [10]:
#nota como los paises no estan como tal disponibles en la base de datos, sino que son los identificadores o indices, lo que implica
#que no se pueden implementar como datos, sino como elementos para señalizar el conjunto (ademas que no tienen los nombres)
#sino que tiene codigos de longitud 3 
#vamos a arreglar eso para poder visualizarlo con mayor facilidad
EmisionesCo2PorCombustibleLiquido

,YR1960,YR1961,YR1962,YR1963,YR1964,YR1965,YR1966,YR1967,YR1968,YR1969,...,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022
economy,,,,,,,,,,,,,,,,,,,,,
ABW,11092.67500,11576.719000,12713.489000,12178.107000,11840.743000,10623.299000,9933.903000,12236.779000,11378.701000,14891.687000,...,861.745,872.746,898.415,883.747,NaN,NaN,NaN,NaN,NaN,NaN
AFE,19042.53518,21102.655839,22561.037401,23749.338206,28592.569339,29917.201248,33119.693685,36439.065408,38531.204628,44788.142407,...,139929.053,163665.544,161509.348,164886.655,NaN,NaN,NaN,NaN,NaN,NaN
AFG,271.35800,293.360000,363.033000,392.369000,476.710000,542.716000,575.719000,557.384000,498.712000,528.048000,...,4785.435,3773.343,4741.431,3476.316,NaN,NaN,NaN,NaN,NaN,NaN
AFW,6934.29700,7422.008000,7623.693000,8401.097000,9699.215000,12115.768000,13193.866000,14701.003000,12867.503000,14701.003000,...,115913.870,124190.289,114733.096,121869.078,NaN,NaN,NaN,NaN,NaN,NaN
AGO,348.36500,311.695000,1008.425000,1034.094000,1041.428000,1012.092000,1334.788000,799.406000,1419.129000,2500.894000,...,17454.920,28265.236,17080.886,17931.630,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1646.483,1587.811,1954.511,1914.174,NaN,NaN,NaN,NaN,NaN,NaN
YEM,58.67200,73.340000,69.673000,80.674000,99.009000,102.676000,99.009000,102.676000,128.345000,135.679000,...,21338.273,20769.888,9372.852,8104.070,NaN,NaN,NaN,NaN,NaN,NaN
ZAF,9075.82500,10373.943000,11100.009000,11705.064000,15298.724000,15133.709000,17231.233000,18888.717000,17311.907000,19926.478000,...,51117.980,56684.486,60721.853,60424.826,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#tomar los paises como elementos de la tabla y no como los indices

EmisionesCo2PorCombustibleLiquido = EmisionesCo2PorCombustibleLiquido.reset_index()
PoblacionTotal = PoblacionTotal.reset_index()
CantidadDeTuristas = CantidadDeTuristas.reset_index()
IngresosPorViajesTuristicos = IngresosPorViajesTuristicos.reset_index()
InflacionPib = InflacionPib.reset_index()

In [12]:
EmisionesCo2PorCombustibleLiquido

,economy,YR1960,YR1961,YR1962,YR1963,YR1964,YR1965,YR1966,YR1967,YR1968,...,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022
0,ABW,11092.67500,11576.719000,12713.489000,12178.107000,11840.743000,10623.299000,9933.903000,12236.779000,11378.701000,...,861.745,872.746,898.415,883.747,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,19042.53518,21102.655839,22561.037401,23749.338206,28592.569339,29917.201248,33119.693685,36439.065408,38531.204628,...,139929.053,163665.544,161509.348,164886.655,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,271.35800,293.360000,363.033000,392.369000,476.710000,542.716000,575.719000,557.384000,498.712000,...,4785.435,3773.343,4741.431,3476.316,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,6934.29700,7422.008000,7623.693000,8401.097000,9699.215000,12115.768000,13193.866000,14701.003000,12867.503000,...,115913.870,124190.289,114733.096,121869.078,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,348.36500,311.695000,1008.425000,1034.094000,1041.428000,1012.092000,1334.788000,799.406000,1419.129000,...,17454.920,28265.236,17080.886,17931.630,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1646.483,1587.811,1954.511,1914.174,NaN,NaN,NaN,NaN,NaN,NaN
262,YEM,58.67200,73.340000,69.673000,80.674000,99.009000,102.676000,99.009000,102.676000,128.345000,...,21338.273,20769.888,9372.852,8104.070,NaN,NaN,NaN,NaN,NaN,NaN
263,ZAF,9075.82500,10373.943000,11100.009000,11705.064000,15298.724000,15133.709000,17231.233000,18888.717000,17311.907000,...,51117.980,56684.486,60721.853,60424.826,NaN,NaN,NaN,NaN,NaN,NaN
264,ZMB,NaN,NaN,NaN,NaN,447.374000,583.053000,553.717000,964.421000,1078.098000,...,2596.236,2922.599,3003.273,2940.934,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#funcion para cambiar los codigos reconocibles por los nombres de los paises 

#utilizaremos la funcion pycountry para cambiar estos codigos de formato alpha3 a nombres

#en caso de que el codigo no existe (ya que el banco mundial crea algunas agrupaciones de paises) retorna el valor original
def ObtenerNombrePais(x):
    try:
        return pycountry.countries.get(alpha_3=x).name
    except:
        return x
    
#Usar la funcion creada para reemplazar los codigos reconocibles por nombres de paises
EmisionesCo2PorCombustibleLiquido.economy = EmisionesCo2PorCombustibleLiquido.economy.apply(lambda x: ObtenerNombrePais(x))
PoblacionTotal.economy = PoblacionTotal.economy.apply(lambda x: ObtenerNombrePais(x))
CantidadDeTuristas.economy = CantidadDeTuristas.economy.apply(lambda x: ObtenerNombrePais(x))
IngresosPorViajesTuristicos.economy = IngresosPorViajesTuristicos.economy.apply(lambda x: ObtenerNombrePais(x))
InflacionPib.economy = InflacionPib.economy.apply(lambda x: ObtenerNombrePais(x))

In [14]:
EmisionesCo2PorCombustibleLiquido

,economy,YR1960,YR1961,YR1962,YR1963,YR1964,YR1965,YR1966,YR1967,YR1968,...,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022
0,Aruba,11092.67500,11576.719000,12713.489000,12178.107000,11840.743000,10623.299000,9933.903000,12236.779000,11378.701000,...,861.745,872.746,898.415,883.747,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,19042.53518,21102.655839,22561.037401,23749.338206,28592.569339,29917.201248,33119.693685,36439.065408,38531.204628,...,139929.053,163665.544,161509.348,164886.655,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,271.35800,293.360000,363.033000,392.369000,476.710000,542.716000,575.719000,557.384000,498.712000,...,4785.435,3773.343,4741.431,3476.316,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,6934.29700,7422.008000,7623.693000,8401.097000,9699.215000,12115.768000,13193.866000,14701.003000,12867.503000,...,115913.870,124190.289,114733.096,121869.078,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,348.36500,311.695000,1008.425000,1034.094000,1041.428000,1012.092000,1334.788000,799.406000,1419.129000,...,17454.920,28265.236,17080.886,17931.630,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1646.483,1587.811,1954.511,1914.174,NaN,NaN,NaN,NaN,NaN,NaN
262,Yemen,58.67200,73.340000,69.673000,80.674000,99.009000,102.676000,99.009000,102.676000,128.345000,...,21338.273,20769.888,9372.852,8104.070,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,9075.82500,10373.943000,11100.009000,11705.064000,15298.724000,15133.709000,17231.233000,18888.717000,17311.907000,...,51117.980,56684.486,60721.853,60424.826,NaN,NaN,NaN,NaN,NaN,NaN
264,Zambia,NaN,NaN,NaN,NaN,447.374000,583.053000,553.717000,964.421000,1078.098000,...,2596.236,2922.599,3003.273,2940.934,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#Cambiar el nombre de las columnas por años y cambiar el nombre economy como pais 
EmisionesCo2PorCombustibleLiquido.columns = ["pais"]+[i[2:] for i in EmisionesCo2PorCombustibleLiquido.columns[1:].values]
PoblacionTotal.columns = ["pais"]+[i[2:] for i in PoblacionTotal.columns[1:].values]
CantidadDeTuristas.columns = ["pais"]+[i[2:] for i in CantidadDeTuristas.columns[1:].values]
IngresosPorViajesTuristicos.columns = ["pais"]+[i[2:] for i in IngresosPorViajesTuristicos.columns[1:].values]
InflacionPib.columns = ["pais"]+[i[2:] for i in InflacionPib.columns[1:].values]

In [16]:
EmisionesCo2PorCombustibleLiquido

,pais,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,11092.67500,11576.719000,12713.489000,12178.107000,11840.743000,10623.299000,9933.903000,12236.779000,11378.701000,...,861.745,872.746,898.415,883.747,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,19042.53518,21102.655839,22561.037401,23749.338206,28592.569339,29917.201248,33119.693685,36439.065408,38531.204628,...,139929.053,163665.544,161509.348,164886.655,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,271.35800,293.360000,363.033000,392.369000,476.710000,542.716000,575.719000,557.384000,498.712000,...,4785.435,3773.343,4741.431,3476.316,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,6934.29700,7422.008000,7623.693000,8401.097000,9699.215000,12115.768000,13193.866000,14701.003000,12867.503000,...,115913.870,124190.289,114733.096,121869.078,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,348.36500,311.695000,1008.425000,1034.094000,1041.428000,1012.092000,1334.788000,799.406000,1419.129000,...,17454.920,28265.236,17080.886,17931.630,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1646.483,1587.811,1954.511,1914.174,NaN,NaN,NaN,NaN,NaN,NaN
262,Yemen,58.67200,73.340000,69.673000,80.674000,99.009000,102.676000,99.009000,102.676000,128.345000,...,21338.273,20769.888,9372.852,8104.070,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,9075.82500,10373.943000,11100.009000,11705.064000,15298.724000,15133.709000,17231.233000,18888.717000,17311.907000,...,51117.980,56684.486,60721.853,60424.826,NaN,NaN,NaN,NaN,NaN,NaN
264,Zambia,NaN,NaN,NaN,NaN,447.374000,583.053000,553.717000,964.421000,1078.098000,...,2596.236,2922.599,3003.273,2940.934,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#Usar nuevamente el pais como indice 
EmisionesCo2PorCombustibleLiquido = EmisionesCo2PorCombustibleLiquido.set_index("pais") 
PoblacionTotal = PoblacionTotal.set_index("pais")
CantidadDeTuristas = CantidadDeTuristas.set_index("pais")
IngresosPorViajesTuristicos = IngresosPorViajesTuristicos.set_index("pais") 
InflacionPib = InflacionPib.set_index("pais")

In [18]:
EmisionesCo2PorCombustibleLiquido

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
pais,,,,,,,,,,,,,,,,,,,,,
Aruba,11092.67500,11576.719000,12713.489000,12178.107000,11840.743000,10623.299000,9933.903000,12236.779000,11378.701000,14891.687000,...,861.745,872.746,898.415,883.747,NaN,NaN,NaN,NaN,NaN,NaN
AFE,19042.53518,21102.655839,22561.037401,23749.338206,28592.569339,29917.201248,33119.693685,36439.065408,38531.204628,44788.142407,...,139929.053,163665.544,161509.348,164886.655,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,271.35800,293.360000,363.033000,392.369000,476.710000,542.716000,575.719000,557.384000,498.712000,528.048000,...,4785.435,3773.343,4741.431,3476.316,NaN,NaN,NaN,NaN,NaN,NaN
AFW,6934.29700,7422.008000,7623.693000,8401.097000,9699.215000,12115.768000,13193.866000,14701.003000,12867.503000,14701.003000,...,115913.870,124190.289,114733.096,121869.078,NaN,NaN,NaN,NaN,NaN,NaN
Angola,348.36500,311.695000,1008.425000,1034.094000,1041.428000,1012.092000,1334.788000,799.406000,1419.129000,2500.894000,...,17454.920,28265.236,17080.886,17931.630,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1646.483,1587.811,1954.511,1914.174,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,58.67200,73.340000,69.673000,80.674000,99.009000,102.676000,99.009000,102.676000,128.345000,135.679000,...,21338.273,20769.888,9372.852,8104.070,NaN,NaN,NaN,NaN,NaN,NaN
South Africa,9075.82500,10373.943000,11100.009000,11705.064000,15298.724000,15133.709000,17231.233000,18888.717000,17311.907000,19926.478000,...,51117.980,56684.486,60721.853,60424.826,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Trabajar con los vacios (seria erroneo asumir que en esos paises los datos con vacios en los años son 0)
#se eliminan los paises con la informacion insuficiente para trabajar 

# (Usar este codigo para decir el porcentaje que faltan datos)
100-EmisionesCo2PorCombustibleLiquido.T.isna().sum()/len(EmisionesCo2PorCombustibleLiquido.columns)*100 

pais
Aruba           90.476190
AFE             90.476190
Afghanistan     90.476190
AFW             90.476190
Angola          90.476190
                  ...    
XKX             14.285714
Yemen           90.476190
South Africa    90.476190
Zambia          84.126984
Zimbabwe        84.126984
Length: 266, dtype: float64

In [20]:
#Primero se tiene en cuenta unicamente los datos desde el 1990
EmisionesCo2PorCombustibleLiquido = EmisionesCo2PorCombustibleLiquido[EmisionesCo2PorCombustibleLiquido.columns[30:]]
PoblacionTotal = PoblacionTotal[PoblacionTotal.columns[30:]]
CantidadDeTuristas = CantidadDeTuristas[CantidadDeTuristas.columns[30:]]
IngresosPorViajesTuristicos = IngresosPorViajesTuristicos[IngresosPorViajesTuristicos.columns[30:]]
InflacionPib = InflacionPib[InflacionPib.columns[30:]]

In [21]:
# (Usar este codigo para decir que faltan datos nuevamente)
# la cantidad de faltantes aumenta porque no hay informacion de los datos mas recientes

100-EmisionesCo2PorCombustibleLiquido.T.isna().sum()/len(EmisionesCo2PorCombustibleLiquido.columns)*100 

pais
Aruba           81.818182
AFE             81.818182
Afghanistan     81.818182
AFW             81.818182
Angola          81.818182
                  ...    
XKX             27.272727
Yemen           81.818182
South Africa    81.818182
Zambia          81.818182
Zimbabwe        81.818182
Length: 266, dtype: float64

In [22]:
#eliminar los que tengan menos del 70% de los datos anuales (esto debido a que algunas tablas solo tienen datos hasta el 2014) 
ProcesadoEmisionesCo2PorCombustibleLiquido = EmisionesCo2PorCombustibleLiquido[100-EmisionesCo2PorCombustibleLiquido.T.isna().sum()/len(EmisionesCo2PorCombustibleLiquido.columns)*100>70].copy()
ProcesadoPoblacionTotal = PoblacionTotal[100-PoblacionTotal.T.isna().sum()/len(PoblacionTotal.columns)*100>70].copy()
ProcesadoCantidadDeTuristas = CantidadDeTuristas[100-CantidadDeTuristas.T.isna().sum()/len(CantidadDeTuristas.columns)*100>70].copy()
ProcesadoIngresosPorViajesTuristicos = IngresosPorViajesTuristicos[100-IngresosPorViajesTuristicos.T.isna().sum()/len(IngresosPorViajesTuristicos.columns)*100>70].copy()
ProcesadoInflacionPib = InflacionPib[100-InflacionPib.T.isna().sum()/len(InflacionPib.columns)*100>70].copy()

In [23]:
100-ProcesadoEmisionesCo2PorCombustibleLiquido.T.isna().sum()/len(ProcesadoEmisionesCo2PorCombustibleLiquido.columns)*100 

pais
Aruba           81.818182
AFE             81.818182
Afghanistan     81.818182
AFW             81.818182
Angola          81.818182
                  ...    
Samoa           81.818182
Yemen           81.818182
South Africa    81.818182
Zambia          81.818182
Zimbabwe        81.818182
Length: 245, dtype: float64

In [24]:
#Calcular los indices o paises en comun que esten en todas las tablas y que sean paises reconocibles mediante el codigo alpha3

#intersect1d es una funcion de la libreria de numpy para hacer interseccion de listas, es decir, para obtener
#los datos que unicamente estan en todas las listas
#por lo cual, unificaremos y uniformizaremos tanto el orden de los paises como la cantidad de los mismos en todas las
# bases de datos

Indices = np.intersect1d(ProcesadoEmisionesCo2PorCombustibleLiquido.index.values,ProcesadoCantidadDeTuristas.index.values)
Indices = np.intersect1d(Indices,ProcesadoIngresosPorViajesTuristicos.index.values)
# Indices = np.intersect1d(Indices,ProcesadoInflacionPib.index.values)
Indices = [i for i in Indices if(len(i)>3)]

In [25]:
#Uniformizar informacion (todas las tablas deben tener los mismos elementos en el mismo orden )

ProcesadoEmisionesCo2PorCombustibleLiquido = ProcesadoEmisionesCo2PorCombustibleLiquido.loc[Indices]
ProcesadoPoblacionTotal = ProcesadoPoblacionTotal.loc[Indices]
ProcesadoCantidadDeTuristas = ProcesadoCantidadDeTuristas.loc[Indices]
ProcesadoIngresosPorViajesTuristicos = ProcesadoIngresosPorViajesTuristicos.loc[Indices]
ProcesadoInflacionPib = ProcesadoInflacionPib.loc[Indices]

In [26]:
# Calcular toneladas de co2 emitidas por persona anualmente debido a combustibles 

EmisionesCo2PorPersonaEnToneladas = ProcesadoEmisionesCo2PorCombustibleLiquido/ProcesadoPoblacionTotal*1000

# Cantidad de turistas por año 

CantidadDeTuristas = ProcesadoCantidadDeTuristas.loc[Indices].copy()

# Calcular ingresos por turista debido a viajes  

IngresosPorTurista = ProcesadoIngresosPorViajesTuristicos/ProcesadoCantidadDeTuristas

#Inflacion 

PorcentajeInflacion = ProcesadoInflacionPib.loc[Indices].copy()

In [27]:
# finalmente vamos a almacenar los datos usando excel, 
writer = pd.ExcelWriter("ResultadosWorldBank.xlsx", engine="xlsxwriter")

EmisionesCo2PorPersonaEnToneladas.to_excel(writer,sheet_name="Co2PorPersonaToneladas")
CantidadDeTuristas.to_excel(writer,sheet_name="Turistas")
IngresosPorTurista.to_excel(writer,sheet_name="IngresosPorTurista")
PorcentajeInflacion.to_excel(writer,sheet_name="PorcentajeInflacion")

writer.close()

In [28]:
# from google.colab import files
# files.download('ResultadosWorldBank.xlsx')